In [34]:
import nltk
import string
import numpy as np
import pandas as pd
import os
from nltk.corpus import stopwords


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [35]:
df = pd.read_csv('/home/user/Desktop/simge_error_analysis/train.csv')
test_df = pd.read_csv('/home/user/Desktop/simge_error_analysis/test.csv')
val_df = pd.read_csv('/home/user/Desktop/simge_error_analysis/val.csv')

In [36]:
df = pd.concat([df, test_df, val_df], ignore_index=True)

In [37]:
df.loc[df.label == 2, 'label'] = 0

In [38]:
df_sent = (df['text'].str.len() > 10)
ne_df = df.loc[df_sent]
df = ne_df
df = df[~df.text.str.startswith('https')]
df = df[~df.text.str.startswith('url : ')]

In [39]:
sent_list = df["text"].tolist()
label_list = df["label"].tolist()

print(len(label_list))
print(label_list[0])

7792
0.0


In [40]:
df.loc[df.label == 2]

,url,text,label


In [41]:
def word_embeddings(path):
    embeddings_index = dict()
    f = open(path)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [42]:
glove_300d = word_embeddings("/home/user/Desktop/timesofindiatrainfiles/glove.6B/glove.6B.300d.txt")

In [43]:
### Create sequence
train_text = df['text']
train_label_list = df['label']
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_text)
def create_sequences_data(pad_len, seq_data=train_text):
    sequences = tokenizer.texts_to_sequences(train_text)
    data = pad_sequences(sequences, maxlen=200)
    return data
data = create_sequences_data(200)

In [44]:
def create_embeddings(word_vector, embedding_dim):
    embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
    for word, index in tokenizer.word_index.items():
        if index > vocabulary_size - 1:
            break
        else:
            embedding_vector = word_vector.get(word)
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
    return embedding_matrix

In [45]:
embedding_matrix_300 = create_embeddings(glove_300d, 300)

In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, label_list, test_size=0.33, random_state=42)

In [47]:
def conv1d_lstm(embedding_matrix, embedding_dim):
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, embedding_dim, input_length=200, weights=[embedding_matrix], trainable=False))
    model_glove.add(Dropout(0.2))
    model_glove.add(Conv1D(64, 5, activation='relu'))
    model_glove.add(MaxPooling1D(pool_size=4))
    model_glove.add(LSTM(200))
    model_glove.add(Dense(1, activation='softmax'))
    model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_glove

In [79]:
def CNN(embedding_matrix, embedding_dim):
    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, weights=[embedding_matrix], input_length=200, trainable=False))
    model.add(Conv1D(64, 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()


    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [72]:
def LSTM_model(embedding_matrix, embedding_dim):
    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, weights=[embedding_matrix], input_length=200, trainable=False))
    model.add(Dropout(0.2))
    model.add(LSTM(40))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [74]:
model_lstm_300 = LSTM_model(embedding_matrix_300, 300)
model_lstm_300.fit(X_train, y_train, batch_size=32, epochs=8)

Epoch 1/8
5220/5220 [==============================] - 27s 5ms/step - loss: 0.4021 - acc: 0.8360
Epoch 2/8
5220/5220 [==============================] - 28s 5ms/step - loss: 0.3101 - acc: 0.8703
Epoch 3/8
5220/5220 [==============================] - 26s 5ms/step - loss: 0.2646 - acc: 0.8921
Epoch 4/8
5220/5220 [==============================] - 25s 5ms/step - loss: 0.2410 - acc: 0.9000
Epoch 5/8
5220/5220 [==============================] - 27s 5ms/step - loss: 0.2261 - acc: 0.9071
Epoch 6/8
5220/5220 [==============================] - 34s 7ms/step - loss: 0.2001 - acc: 0.9201
Epoch 7/8
5220/5220 [==============================] - 25s 5ms/step - loss: 0.1808 - acc: 0.9268
Epoch 8/8
5220/5220 [==============================] - 26s 5ms/step - loss: 0.1589 - acc: 0.9352


In [76]:
model_lstm_300.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
5220/5220 [==============================] - 23s 5ms/step - loss: 0.1422 - acc: 0.9439
Epoch 2/2
5220/5220 [==============================] - 24s 5ms/step - loss: 0.1221 - acc: 0.9525


In [82]:
model_cnn_300 = CNN(embedding_matrix_300, 300)
model_cnn_300.fit(X_train, y_train, batch_size=32, epochs=3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 200, 300)          6000000   
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 196, 64)           96064     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               16640     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 257       
Total params: 6,112,961
Trainable params: 112,961
Non-trainable params: 6,000,000
_________________________________________________________________
Epoch 1/3
6233/6233 [==============================] - 21s 3ms/step - loss: 0.3306 - acc: 0.8639
Epoch 2/3
6233/6233 [========

In [55]:
model_conv1d_lstm_300 = conv1d_lstm(embedding_matrix_300, 300)
model_conv1d_lstm_300.fit(data, train_label_list, batch_size=32, epochs=2)

Epoch 1/2
 768/7792 [=>............................] - ETA: 50s - loss: 13.2853 - acc: 0.1667

KeyboardInterrupt: 

In [34]:
len(test_data)

1180

In [83]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


prediction = model_cnn_300.predict(X_test)
prediction = [1 if pred>0.5 else 0 for pred in prediction]

print("F1 Score is : " + str(f1_score(y_test, prediction)))
print("Accuracy Score is : " + str(accuracy_score(y_test, prediction)))
print("Precision Score is : " + str(precision_score(y_test, prediction)))
print("Recall Score is : " + str(recall_score(y_test, prediction)))


F1 Score is : 0.5793991416309013
Accuracy Score is : 0.8742783835792175
Precision Score is : 0.7068062827225131
Recall Score is : 0.4909090909090909


In [77]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


lstm_prediction = model_lstm_300.predict(X_test)
lstm_prediction = [1 if pred>0.5 else 0 for pred in lstm_prediction]


print("F1 Score is : " + str(f1_score(y_test, lstm_prediction)))
print("Accuracy Score is : " + str(accuracy_score(y_test, lstm_prediction)))
print("Precision Score is : " + str(precision_score(y_test, lstm_prediction)))
print("Recall Score is : " + str(recall_score(y_test, lstm_prediction)))


F1 Score is : 0.6201550387596899
Accuracy Score is : 0.885692068429238
Precision Score is : 0.7339449541284404
Recall Score is : 0.5369127516778524


In [ ]:
prediction = model_lstm_100.predict_classes(test_data)

print("F1 Score is : " + str(f1_score(test_label_list, prediction, average='weighted')))
print("Accuracy Score is : " + str(accuracy_score(test_label_list, prediction)))


In [33]:
def test_model(model, name):
    print(name + " Statistics :")
    
    prediction = model.predict(X_test)
    prediction = [1 if pred>0.5 else 0 for pred in prediction]

    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score

    print("F1 Score is : " + str(f1_score(y_test, prediction)))
    print("Accuracy Score is : " + str(accuracy_score(y_test, prediction)))
    print("Precision Score is : " + str(precision_score(y_test, prediction)))
    print("Recall Score is : " + str(recall_score(y_test, prediction)))

In [45]:
test_model(model_cnn_100, "model_cnn_100")

model_cnn_100 Statistics :


ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [177]:
model_glove = create_model_Conv1d_LSTM(embedding_matrix_300, 300)

In [165]:
model_glove.fit(X_train, np.array(y_train), validation_split=0.0, batch_size=8, epochs=8)

Epoch 1/4
886/886 [==============================] - 12s 14ms/step - loss: 0.5636 - acc: 0.7336
Epoch 2/4
886/886 [==============================] - 8s 9ms/step - loss: 0.3890 - acc: 0.8296
Epoch 3/4
886/886 [==============================] - 8s 9ms/step - loss: 0.3025 - acc: 0.8736
Epoch 4/4
886/886 [==============================] - 8s 9ms/step - loss: 0.1221 - acc: 0.9639


In [201]:
prediction = pipeline.predict(X_test)
prediction = [1 if pred>0.5 else 0 for pred in prediction]

In [85]:
len(df.loc[df.label == 1])

1104